In [1]:
# %load_ext autoreload
# %autoreload 2
# %matplotlib inline
# import matplotlib.pyplot as plt
import sys; sys.path.append('../')
from misc import h5file

import numpy as np
from numpy.random import default_rng
import scipy.io as sio

import torch, sympytorch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
from siren_pytorch import SirenNet

import pysindy as ps

from sympy import symbols, simplify, lambdify
from mathparser import math_eval
from varname import nameof

import sys; sys.path.append('../optimizers/')
from Adan import Adan
from FNO_Adam import Adam

MAIN_SEED = 1234

In [2]:
X_pre, best_subsets, y_pre = h5file(file_path="../PMS_data/burgers_pms.h5", 
                                    mode='r', return_dict=False)

['X_pre', 'best_subsets', 'y_pre']


In [3]:
un = sio.loadmat("../RDAE_data/l21rdae_noisy40_burgers_0.5.mat")["rdae_denoised_un"].real

In [4]:
data = sio.loadmat('../Datasets/burgers.mat')
x = (data['x'][0]).real
t = (data['t'][:,0]).real
dt = t[1]-t[0]; dx = x[2]-x[1]
X, T = np.meshgrid(x, t)
XT = np.asarray([X, T]).T
del data

In [5]:
import yaml
with open("../PMS_data/burgers_pms_feature_names.yaml", 'r') as f:
    config = yaml.load(f, yaml.Loader)
f.close()
encoded_feature_names = config["encoded_feature_names"]
encoded_pde_names = config["encoded_pde_names"]

In [6]:
encoded_pde_names

['u*u_1',
 'u_11+u*u_1',
 'u_11+u*u_1+u*u_11',
 'u_11+u*u_1+u*u_11+u*u*u_11',
 'u+u_11+u*u_1+u*u_11+u*u*u_11',
 'u+u_11+u*u_1+u*u*u_1+u*u_11+u*u*u_11',
 'u+u*u+u_11+u*u_1+u*u*u_1+u*u_11+u*u*u_11',
 'u+u*u+u_1+u_11+u*u_1+u*u*u_1+u*u_11+u*u*u_11']

In [7]:
def log_like_value(prediction, ground):                                                                                                               
    nobs = float(ground.shape[0])
    nobs2 = nobs / 2.0
    ssr = np.sum(np.abs(ground - prediction)**2)
    llf = -nobs2 * np.log(2 * np.pi) - nobs2 * np.log(ssr / nobs) - nobs2
    return llf

def BIC_AIC(prediction, ground, nparams, reg_func = lambda x: x):
    nparams = reg_func(nparams)
    llf = log_like_value(prediction, ground)
    return -2*llf + np.log(ground.shape[0])*nparams, -2*llf + 2*nparams

In [8]:
class PhysicalConstraintCalculator(nn.Module):
    def __init__(self, symbolic_module, basic_vars, init_coefficients=None, learnable_coefficients=False):
        super(PhysicalConstraintCalculator, self).__init__()
        self.symbolic_module = symbolic_module
        self.basic_vars = basic_vars
        
        self.coefficients = init_coefficients
        self.learnable_coefficients = learnable_coefficients

        if self.coefficients is None:
            self.coefficients = torch.ones(len(symbolic_module.sympy())).float()
        else:
            self.coefficients = torch.tensor(data=self.coefficients).float()
        self.coefficients = nn.Parameter(self.coefficients).requires_grad_(self.learnable_coefficients)
        
        # printing
        if self.learnable_coefficients: print("Learnable coefficients:", self.coefficients)
        else: print("NOT learnable coefficients:", self.coefficients)
        print(symbolic_module.sympy())
        print("Basic variables:", self.basic_vars)

    def set_learnable_coefficients(self, learn):
        self.coefficients.requires_grad_(learn)
    
    def forward(self, input_dict):
        return self.symbolic_module(**input_dict)

In [9]:
class Sine(nn.Module):
    def __init__(self, ):
        super(Sine, self).__init__()
    def forward(self, x):
        return torch.sin(x)

class TorchMLP(nn.Module):
    def __init__(self, dimensions, bias=True, activation_function=nn.Tanh(), bn=None, dropout=None):
        super(TorchMLP, self).__init__()
        # setup ModuleList
        self.model  = nn.ModuleList()
        for i in range(len(dimensions)-1):
            self.model.append(nn.Linear(dimensions[i], dimensions[i+1], bias=bias))
            if bn is not None and i!=len(dimensions)-2:
                self.model.append(bn(dimensions[i+1]))
                if dropout is not None:
                    self.model.append(dropout)
            if i==len(dimensions)-2: break
            self.model.append(activation_function)
        # weight init
        self.model.apply(self.xavier_init)

    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.0)

    def forward(self, x):
        for i, l in enumerate(self.model): 
            x = l(x)
        return x

In [10]:
class DeepONet(nn.Module):
    def __init__(self, solver, sensor_network, physics_calculator, lb, ub, 
                 domain_dimension=None, weak_pde_lib=None, effective_indices=None):
        super(DeepONet, self).__init__()
        self.solver = solver
        self.solver_last_layer = nn.Linear(50,1,bias=True)
        torch.nn.init.xavier_uniform_(self.solver_last_layer.weight)
        self.sensor_network = sensor_network
        self.m0 = nn.Parameter(torch.tensor([1.0]).float())
        self.physics_calculator = physics_calculator
        self.lb = lb
        self.ub = ub
        # Only to use weak_loss
        # spatial x temporal
        self.domain_dimension = domain_dimension
        self.weak_pde_lib = weak_pde_lib
        self.effective_indices = effective_indices
        self.weak_coeff_buffer = None
        
    def forward(self, x, t, u0):
        loc = self.solver(self.input_normalize(torch.cat([x, t],  dim=-1)))
        G = self.sensor_network(u0)
        # torch.einsum("bi,ni->nb", G, loc)+self.b0
        return self.solver_last_layer(torch.sin(loc))+(self.m0*torch.matmul(loc, G.T))

    def calculate_physics(self, x, t, u0):
        u = self.forward(x, t, u0)
        u_t = self.gradients(u, t)[0]
        u_1 = self.gradients(u, x)[0]
        u_11 = self.gradients(u_1, x)[0]
        physics = self.physics_calculator({nameof(u):u, 
                                           nameof(u_1):u_1, 
                                           nameof(u_11):u_11})
        
        return u, u_t, physics
    
    def loss(self, x, t, u0, y_input):
        u, u_t, physics = self.calculate_physics(x, t, u0)
        coeff = self.physics_calculator.coefficients
        physics = (physics*coeff).sum(axis=-1)
        mse = F.mse_loss(u, y_input, reduction='mean')
        l_eq = F.mse_loss(u_t, physics, reduction='mean')
        return torch.add(mse, l_eq)
    
    def weak_loss(self, x, t, u0, y_input):
        u, u_t, physics = self.calculate_physics(x, t, u0)
        coeff = torch.tensor(self.weak_coefficients(u)).float()
        physics = (physics*coeff).sum(axis=-1)
        mse = F.mse_loss(u, y_input, reduction='mean')
        l_eq = F.mse_loss(u_t, physics, reduction='mean')
        return torch.add(mse, l_eq)
    
    def weak_form(self, u):
        pred = u.reshape(self.domain_dimension[1], 
                         self.domain_dimension[0]).T.detach().numpy()
        pred = np.expand_dims(pred,-1)
        X_weak = self.weak_pde_lib.fit_transform(pred)
        y_weak = self.weak_pde_lib.convert_u_dot_integral(pred)
        return X_weak, y_weak
    
    def weak_coefficients(self, u):
        np.random.seed(0)
        X_weak, y_weak = self.weak_form(u)
        X_weak = X_weak[:, self.effective_indices]
        self.weak_coeff_buffer = np.linalg.lstsq(X_weak, y_weak, rcond=None)[0].flatten()
        return self.weak_coeff_buffer
    
    def gradients(self, func, x):
        return grad(func, x, create_graph=True, retain_graph=True, 
                    grad_outputs=torch.ones(func.shape))

    def input_normalize(self, inp):
        return -1.0+2.0*(inp-self.lb)/(self.ub-self.lb)

In [11]:
rng = default_rng(seed=0)
# sampled_indices_x = rng.choice(len(x), size=int(len(x)//(2)), replace=False)
# sampled_indices_t = rng.choice(len(t), size=int(len(t)//(2)), replace=False)
sampled_indices_x = np.array([i for i in range(len(x)) if i%2==0])
sampled_indices_t = np.array([i for i in range(len(t)) if i%2==0]) 
domain_dimension = len(sampled_indices_x), len(sampled_indices_t)

In [12]:
np.random.seed(MAIN_SEED);
torch.manual_seed(MAIN_SEED);

In [13]:
XX = X[sampled_indices_t, :][:, sampled_indices_x]
TT = T[sampled_indices_t, :][:, sampled_indices_x]
XXTT = XT[sampled_indices_x, :, :][:, sampled_indices_t, :]

In [14]:
K = 3000; diff_order = 2
weak_pde_lib = ps.WeakPDELibrary(library_functions=[lambda x:x, lambda x: x*x], 
                                 function_names=[lambda x:x, lambda x: x+x], 
                                 derivative_order=diff_order, p=diff_order, 
                                 spatiotemporal_grid=XXTT, 
                                 include_bias=False, is_uniform=True, K=K # new random K points in every calls to the ps.WeakPDELibrary
                                )

In [15]:
X_train = np.hstack((XX.flatten()[:,None], TT.flatten()[:,None]))
y_train = un.T[sampled_indices_t, :][:, sampled_indices_x].flatten()[:,None]
lb = torch.tensor(X_train.min(axis=0)).float().requires_grad_(False)
ub = torch.tensor(X_train.max(axis=0)).float().requires_grad_(False)

In [16]:
u0 = y_train[(X_train[:, 1:2] == 0.0).flatten(), :]
u0 = torch.tensor(u0).float().reshape(1,-1)

In [17]:
del XX, TT, XXTT, X, T, XT

In [18]:
# Converting to tensors
X_train = torch.tensor(X_train).float().requires_grad_(True)
y_train = torch.tensor(y_train).float().requires_grad_(False)
X_train.shape, y_train.shape

(torch.Size([6528, 2]), torch.Size([6528, 1]))

In [19]:
com = 2; com = max(com, 1)
effective_indices = np.where(best_subsets[com-1]>0)[0].tolist()
init_coefficients = np.linalg.lstsq(X_pre[:, effective_indices], 
                                    y_pre, rcond=None)[0].flatten()
mod, basic_vars = math_eval(encoded_pde_names[com-1], 
                            return_torch=True, split_by_addition=True)
init_coefficients, mod

(array([ 0.09517879, -1.0106909 ], dtype=float32),
 SymPyModule(expressions=(u_11, u*u_1)))

In [20]:
# bias init at 0.01 | SIREN

solver = TorchMLP([2,50,50,50,50], bn=None, activation_function=Sine())
# solver = SirenNet(dim_in=2, dim_hidden=50, dim_out=1, num_layers = 4, 
#                   w0_initial = 30.)

sensor_network = TorchMLP([u0.shape[-1],50,50], bn=None, activation_function=Sine())

physics_calculator = PhysicalConstraintCalculator(symbolic_module=mod, 
                                                  basic_vars=basic_vars, 
                                                  init_coefficients=init_coefficients, 
                                                  learnable_coefficients=True)

Learnable coefficients: Parameter containing:
tensor([ 0.0952, -1.0107], requires_grad=True)
[u_11, u*u_1]
Basic variables: ['u', 'u_1', 'u_11']


In [21]:
don = DeepONet(solver, sensor_network, physics_calculator, 
                lb, ub, domain_dimension, 
                weak_pde_lib, effective_indices)

In [22]:
don.physics_calculator.set_learnable_coefficients(False)
lbfgs = torch.optim.LBFGS(don.parameters(), lr=0.1, 
                          max_iter=500, max_eval=500, history_size=300, 
                          line_search_fn='strong_wolfe')
epochs = 100
don.train()

for i in range(epochs):
    def closure():
        if torch.is_grad_enabled(): 
            lbfgs.zero_grad()
        l = don.loss(X_train[:, 0:1], X_train[:, 1:2], u0, y_train)
        if l.requires_grad: 
            l.backward()
        return l

    lbfgs.step(closure)

    # calculate the loss again for monitoring
    if (i%50)==0 or i==epochs-1:
        l = closure()
        print("Epoch {}: ".format(i), l.item())

Epoch 0:  0.0028746635653078556
Epoch 50:  0.0023274405393749475
Epoch 99:  0.0023274405393749475


In [23]:
don.eval()
pred = don(X_train[:, 0:1], X_train[:, 1:2], u0).detach().numpy()
BIC_AIC(pred, y_train.detach().numpy(), com)

(-21082.6181447266, -21096.185856519885)

In [24]:
# torch.save(don.state_dict(), "../PMS_weights/don_mlp_sine.pth")

#### weak

In [25]:
### using lbfgs ###
print("using lbfgs...")
epochs = 100
don.train()
don.physics_calculator.set_learnable_coefficients(False)
lbfgs2 = torch.optim.LBFGS(don.parameters(), lr=0.1, 
                          max_iter=500, max_eval=500, history_size=300, 
                          line_search_fn='strong_wolfe')

for i in range(epochs):
    def closure2():
        if torch.is_grad_enabled(): 
            lbfgs2.zero_grad()
        l = don.weak_loss(X_train[:, 0:1], X_train[:, 1:2], u0, y_train)
        if l.requires_grad: 
            l.backward()
        return l

    lbfgs2.step(closure2)

    # calculate the loss again for monitoring
    if (i%50)==0 or i==epochs-1:
        l = closure2()
        print("Epoch {}: ".format(i), l.item())

don.eval()
pred = don(X_train[:, 0:1], X_train[:, 1:2], u0).detach().numpy()
print(BIC_AIC(pred, y_train.detach().numpy(), com))
print(don.physics_calculator.coefficients.detach().numpy())
print(don.weak_coeff_buffer)

### using non-lbfgs ###
print("using non-lbfgs...")
epochs = 2000 # 1000 to 2000 Okay
don.train()
don.physics_calculator.set_learnable_coefficients(False)
# optimizer = Adam(pinn.parameters(), lr=1e-5, weight_decay=1e-4)
optimizer = Adan(don.parameters(), lr=1e-5, weight_decay=1e-2)

for i in range(epochs):
    optimizer.zero_grad()
    l = don.weak_loss(X_train[:, 0:1], X_train[:, 1:2], u0, y_train)
    l.backward()
    optimizer.step()
    if (i%50)==0 or i==epochs-1:
        l = don.weak_loss(X_train[:, 0:1], X_train[:, 1:2], u0, y_train)
        print("Epoch {}: ".format(i), l.item())

using lbfgs...
Epoch 0:  0.002327537629753351
Epoch 50:  0.002327489433810115
Epoch 99:  0.002327489433810115
(-21083.277247995444, -21096.84495978873)
[ 0.09517879 -1.0106909 ]
[ 0.09805153 -1.00114632]
using non-lbfgs...
Epoch 0:  0.0023282512556761503
Epoch 50:  0.002327435417100787
Epoch 100:  0.0023273909464478493
Epoch 150:  0.002327350666746497
Epoch 200:  0.0023273255210369825
Epoch 250:  0.002327294321730733
Epoch 300:  0.0023272705730050802
Epoch 350:  0.0023272496182471514
Epoch 400:  0.002327226335182786
Epoch 450:  0.0023272098042070866
Epoch 500:  0.0023271902464330196
Epoch 550:  0.0023271769750863314
Epoch 600:  0.0023271599784493446
Epoch 650:  0.0023271481040865183
Epoch 700:  0.00232713483273983
Epoch 750:  0.002327118767425418
Epoch 800:  0.0023271040990948677
Epoch 850:  0.002327089197933674
Epoch 900:  0.002327076392248273
Epoch 950:  0.002327065449208021
Epoch 1000:  0.002327053342014551
Epoch 1050:  0.0023270423989742994
Epoch 1100:  0.0023270335514098406
Epoch 

#### learn (pinn.physics_calculator.set_learnable_coefficients(True))

In [26]:
### using non-lbfgs ###
# epochs = 1000
# pinn.train()
# pinn.physics_calculator.set_learnable_coefficients(True)
# # optimizer = Adam(pinn.parameters(), lr=1e-5, weight_decay=1e-4)
# optimizer = Adan(pinn.parameters(), lr=1e-5, weight_decay=1e-2)

# for i in range(epochs):
#     optimizer.zero_grad()
#     l = pinn.loss(X_train[:, 0:1], X_train[:, 1:2], y_train)
#     l.backward()
#     optimizer.step()
#     if (i%50)==0 or i==epochs-1:
#         l = pinn.loss(X_train[:, 0:1], X_train[:, 1:2], y_train)
#         print("Epoch {}: ".format(i), l.item())

### using lbfgs ###
# epochs = 100
# pinn.train()
# pinn.physics_calculator.set_learnable_coefficients(True)
# lbfgs2 = torch.optim.LBFGS(pinn.parameters(), lr=0.1, 
#                           max_iter=500, max_eval=500, history_size=300, 
#                           line_search_fn='strong_wolfe')

# for i in range(epochs):
#     def closure2():
#         if torch.is_grad_enabled(): 
#             lbfgs2.zero_grad()
#         l = pinn.loss(X_train[:, 0:1], X_train[:, 1:2], y_train)
#         if l.requires_grad: 
#             l.backward()
#         return l

#     lbfgs2.step(closure2)

#     # calculate the loss again for monitoring
#     if (i%50)==0 or i==epochs-1:
#         l = closure2()
#         print("Epoch {}: ".format(i), l.item())

#### eval

In [36]:
don.eval()
pred = don(X_train[:, 0:1], X_train[:, 1:2], u0).detach().numpy()
print(BIC_AIC(pred, y_train.detach().numpy(), com))
print(don.physics_calculator.coefficients.detach().numpy())
print(don.weak_coeff_buffer)

(-21083.125363570798, -21096.693075364085)
[ 0.09517879 -1.0106909 ]
[ 0.09903233 -1.00017205]


In [37]:
# torch.save(don.state_dict(), "../PMS_weights/don_mlp_sine_weak.pth")

In [38]:
def percent_coeff_error(pred):
    ground = np.array([0.1, -1])
    errs = 100*np.abs(np.array(pred)-ground)/np.abs(ground)
    return errs.mean(), errs.std()
print(percent_coeff_error([ 0.09517879, -1.0106909 ]))
print(percent_coeff_error([ 0.0990209, -1.00016034]))
print(percent_coeff_error([ 0.09898001, -0.99915481]))
print(percent_coeff_error([ 0.09899466, -0.99912456]))
print(percent_coeff_error([0.09903233, -1.00017205]))

(2.9451500000000004, 1.8760600000000056)
(0.49756700000000986, 0.48153300000000065)
(0.5522545000000086, 0.4677355000000035)
(0.5464420000000011, 0.45889800000000613)
(0.49243750000000086, 0.47523250000000294)


#### Notes
    - 3 main files are required.

In [39]:
# torch.save(pinn.state_dict(), "../PMS_weights/pinn_mlp_sine_weak.pth")
# (-21077.95679626977, -21091.524508063056)
# [[ 0.09809884]
#  [-1.00043748]]

In [40]:
# nn.Tanh(): (-20637.284432281755, -20650.81253882045)
# Sine(): (-20681.104197432018, -20694.632303970713)
# SirenNet(dim_in = 2, dim_hidden = 50, dim_out = 1, num_layers = 4, w0_initial = 30.): (-21585.25976244083, -21598.787868979525)